In [18]:
!pip install spotipy

In [15]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

client_id='e08812d0ad874b53bb01e49ada6166ad'
client_secret='9085378d43d24d6c823a92c0f2cb03b5'

# # 민진님
# client_id = "7a98ceda33624c9bb15bf7f877176e50"
# client_secret = "74fb60bce9d048e3a9faa52427e6d0f5"

# # 세인님
# client_id = "05c96c56746e49b9b26e2ccf9837c7b7"
# client_secret = "98238151cd5e46df93049946ddd3673a"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

플레이리스트 가져오는 코드

In [4]:
import pandas as pd
import time
import os
import datetime

start_time = time.time()

search_query = 'k-pop'
search_results = sp.search(search_query, type='playlist', limit=50)

playlist_ids = [playlist['id'] for playlist in search_results['playlists']['items']]

data = []
for playlist_id in playlist_ids:
    results = sp.playlist(playlist_id)
    for idx, item in enumerate(results['tracks']['items']):
        track = item['track']
        if track is None:
            continue
        data.append([results['id'], results['name'], results['owner']['id'], track['artists'][0]['id'], track['artists'][0]['name'], track['id'], track['name']])

df = pd.DataFrame(data, columns=['playlist_id', 'playlist_name', 'user_id', 'Artist ID', 'Artist', 'Track ID', 'Track'])

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") 
df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/API_playlist/playlist_info/{search_query}_audio_features_{timestamp}.csv', index=False, encoding='utf-8-sig')

end_time = time.time() 
print(f"작업 시작 시간: {start_time}") 
print(f"작업 종료 시간: {end_time}")  
print(f"작업 소요 시간: {end_time - start_time} 초") 


작업 시작 시간: 1701749948.530795
작업 종료 시간: 1701749964.1622894
작업 소요 시간: 15.631494283676147 초


In [17]:
import pandas as pd
import time
import os
import datetime

start_time = time.time()

search_query = 'k-pop'
search_results = sp.search(search_query, type='playlist', limit=50)
playlist_ids = [playlist['id'] for playlist in search_results['playlists']['items']]

data = []

artist_id = []
artist_name = []
artist_popularity = []
artist_followers = []

track_id = []
track_name = []
release_date = []
track_popularity = []

album_image = []

track_features_list = []

iteration = 0 

for playlist_id in playlist_ids:
    iteration += 1  
    results = sp.playlist(playlist_id)
    
    for idx, item in enumerate(results['tracks']['items'][0:3]):  
        track = item['track']
        if track is None:
            continue
          
        data.append([results['id'], results['name'], results['owner']['id'], track['artists'][0]['id'], track['artists'][0]['name'], track['id'], track['name']])
        
        artist = sp.artist(track['artists'][0]['id']) 
        artist_id.append(artist['id'])
        artist_name.append(artist['name'])
        artist_popularity.append(artist['popularity'])
        artist_followers.append(artist['followers']['total'])

        track_id.append(track['id'])
        track_name.append(track['name'])
        release_date.append(track['album']['release_date'])
        track_popularity.append(track['popularity'])

        try:
            album_image.append(track['album']['images'][1]['url'])  
        except IndexError:
            album_image.append(None) 

        track_features = sp.audio_features([track['id']])
        if track_features:
            track_features_list.append(track_features[0])
        else:
            track_features_list.append(None)

    print("for문 다시 시작전 3분 대기")
    time.sleep(10)


    df = pd.DataFrame(data, columns=['playlist_id', 'playlist_name', 'user_id', 'Artist ID', 'Artist', 'Track ID', 'Track'])
    timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/API_playlist/playlist_info/{search_query}_audio_features_{timestamp}_{iteration}.csv', index=False, encoding='utf-8-sig')

    artist_df = pd.DataFrame({'artist_id': artist_id, 'artist_name': artist_name, 'artist_popularity': artist_popularity, 'artist_followers': artist_followers})
    artist_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/API_playlist/playlist_artist/{search_query}_artist_{timestamp}_{iteration}.csv', index=False)

    track_df = pd.DataFrame({'track_id': track_id, 'track_name': track_name, 'release_date': release_date, 'track_popularity': track_popularity, 'album_image': album_image})
    track_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/API_playlist/playlist_track/{search_query}_track_{timestamp}_{iteration}.csv', index=False)

    track_features_df = pd.DataFrame(track_features_list)
    track_features_df = track_features_df.drop(columns=['type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
    track_features_df.to_csv(f'/home/ubuntu/project/4.5HZ/Data/API_playlist/playlist_audio/{search_query}_audio_features_{timestamp}_{iteration}.csv', index=False)

end_time = time.time() 
print(f"작업 시작 시간: {start_time}") 
print(f"작업 종료 시간: {end_time}")  
print(f"작업 소요 시간: {end_time - start_time} 초") 


KeyboardInterrupt: 

In [5]:
df = pd.read_csv(f'/home/ubuntu/project/4.5HZ/Data/API_playlist/playlist_info/{search_query}_audio_features_{timestamp}.csv', encoding='utf-8-sig')

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3921 entries, 0 to 3920
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   playlist_id    3921 non-null   object
 1   playlist_name  3921 non-null   object
 2   user_id        3921 non-null   object
 3   Artist ID      3921 non-null   object
 4   Artist         3921 non-null   object
 5   Track ID       3921 non-null   object
 6   Track          3921 non-null   object
dtypes: object(7)
memory usage: 214.6+ KB


,playlist_id,playlist_name,user_id,Artist ID,Artist,Track ID,Track
0,2EoheVFjqIxgJMb8VnDRtZ,BEST KPOP 2023 - TOP HITS,11156981936,6HaGTQPmzraVmaVxvz6EUc,Jung Kook,7x9aauaA9cu6tyfpHnqDLo,Seven (feat. Latto) (Explicit Ver.)
1,2EoheVFjqIxgJMb8VnDRtZ,BEST KPOP 2023 - TOP HITS,11156981936,6HvZYsbFfjnjFrWF950C9d,NewJeans,5sdQOyqq2IDhvmx2lHOpwd,Super Shy
2,2EoheVFjqIxgJMb8VnDRtZ,BEST KPOP 2023 - TOP HITS,11156981936,4SpbR6yFEvexJuaBpgAU5p,LE SSERAFIM,74X2u8JMVooG2QbjRxXwR8,Perfect Night
3,2EoheVFjqIxgJMb8VnDRtZ,BEST KPOP 2023 - TOP HITS,11156981936,250b0Wlc5Vk0CoUsaCY84M,JENNIE,6gcuJpHu0Ey30D5WR76y98,You & Me
4,2EoheVFjqIxgJMb8VnDRtZ,BEST KPOP 2023 - TOP HITS,11156981936,1SIocsqdEefUTE6XKGUiVS,BABYMONSTER,3VBj0lzjmhTzVFPEDOjNCG,BATTER UP


In [6]:
df_artist_id = df[['Artist ID']].rename(columns={'Artist ID': 'artist_id_2'})

df_artist_id.head()

,artist_id_2
0,6HaGTQPmzraVmaVxvz6EUc
1,6HvZYsbFfjnjFrWF950C9d
2,4SpbR6yFEvexJuaBpgAU5p
3,250b0Wlc5Vk0CoUsaCY84M
4,1SIocsqdEefUTE6XKGUiVS


In [ ]:
artist_id_list = df_artist_id['artist_id_2'].tolist()

for artist_id_3 in artist_id_list:
    print(artist_id_3)

6HaGTQPmzraVmaVxvz6EUc
6HvZYsbFfjnjFrWF950C9d
4SpbR6yFEvexJuaBpgAU5p
250b0Wlc5Vk0CoUsaCY84M
1SIocsqdEefUTE6XKGUiVS
28ot3wh4oNmoFOdVajibBl
68KmkJeZGfwe1OUaivBa2L
5t5FqBwTcgKTaWmfEbwQY9
1z4g3DjTBBZKhvAroFlhOM
6YVMFz59CuY7ngCxTxjpxE
4tmWAc4dENNTV6b4lgcpdg
6L2Aqx4Tk8Mlm0tjUQQTx3
2dIgFjalVxs4ThymZ67YCE
6Xg22wJOAcnvPUfk5WvODH
7uGeDBa1LJ7T1X4fpl8mwk
6RHTUrRF63xao58xh9FXYJ
1RrRhvizF5SjIY8rIf2bMn
7Lq3yAtwi0Z7zpxEwbQQNZ
0ghlgldX5Dd6720Q3qFyQB
7nqOGRxlXj7N2JYbgNEjYH
4tmWAc4dENNTV6b4lgcpdg
13rF01aOogvnkuQXOlgTW8
3RWZfQory9Hv1kADBi6bHJ
47mIJdHORyRerp4os813jD
6OwKE9Ez6ALxpTaKcT5ayv
4tmWAc4dENNTV6b4lgcpdg
6L2Aqx4Tk8Mlm0tjUQQTx3
6HaGTQPmzraVmaVxvz6EUc
3vNisYibo8wFyD4wxIbSbn
2AfmfGFbe0A0WsTYm0SDTx
7f4ignuCJhLXfZ9giKT7rH
4tmWAc4dENNTV6b4lgcpdg
7zYj9S9SdIunYCfSm7vzAR
7bmYpVgQub656uNTu6qGNQ
6YVMFz59CuY7ngCxTxjpxE
7F1iAHRYxR3MY7yAEuFqgL
2KC9Qb60EaY0kW4eH68vr3
1RrRhvizF5SjIY8rIf2bMn
0LOK81e9H5lr61HlGGHqwA
5YfHsKbly0kK2dYnSkEWKx
01XYiBYaoMJcNhPokrg0l0
6HaGTQPmzraVmaVxvz6EUc
0ghlgldX5Dd6720Q3qFyQB
6HvZYsbFfjn

In [40]:
df_track_id = df[['Track ID']].rename(columns={'Track ID': 'track_id'})

df_track_id.head()

,track_id
0,7x9aauaA9cu6tyfpHnqDLo
1,5sdQOyqq2IDhvmx2lHOpwd
2,74X2u8JMVooG2QbjRxXwR8
3,6gcuJpHu0Ey30D5WR76y98
4,3VBj0lzjmhTzVFPEDOjNCG
